# House Prices - Advanced Regression Techniques

Датасет взят из https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques


In [1]:
#Импортируем библиотеки
import pandas as pd
import numpy as np
import seaborn as sns


In [2]:
#Загружаем данные из файла
df = pd.read_csv("data/house_prices/train.csv", index_col=0)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


SalePrice - Цена продажи. Таргетная переменная, которую необходимо предсказать.

В остальных колонках содержится информация о доме(площадь лота, год постройки, тип дороги у дома и тд) или о сделке(месяц,год продажи,тип сделки)

# Подготовка данных

## Вещественные признаки

In [3]:
# всего строк
df.shape

(1460, 80)

In [4]:
# выделим числовые колонки и посмотрим их описание
numeric_columns = df.loc[:,df.dtypes!=object].columns
df.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


В описании датасета для большинства случаев отсутствующее значение == отсутствию данного признака у обьекта(как например, отсутствует подвал, отсутствует земля перед домом и тд)

Для вещественных колонок заполним пропущенные значения нулями

In [5]:
for col in numeric_columns:
    df[col] = df[col].fillna(0)

Проверим результат

In [6]:
df.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,57.623288,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.117123,443.639726,46.549315,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,34.664304,9981.264932,1.382997,1.112799,30.202904,20.645407,180.731373,456.098091,161.319273,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,0.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,42.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,63.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,79.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,164.250000,712.250000,0.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


Посмотрим корреляцию между признаками

In [7]:
df[numeric_columns].corr()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
MSSubClass,1.000000,-0.215023,-0.139781,0.032628,-0.059316,0.027850,0.040581,0.023573,-0.069836,-0.065649,...,-0.012579,-0.006100,-0.012037,-0.043825,-0.026030,0.008283,-0.007683,-0.013585,-0.021407,-0.084284
LotFrontage,-0.215023,1.000000,0.100739,0.176561,-0.053457,0.036853,0.078686,0.105010,0.076670,-0.009312,...,-0.016780,0.069605,0.027366,0.023499,0.022969,0.114106,-0.059606,0.018942,-0.012094,0.209624
LotArea,-0.139781,0.100739,1.000000,0.105806,-0.005636,0.014228,0.013788,0.103321,0.214103,0.111170,...,0.171698,0.084774,-0.018340,0.020423,0.043160,0.077672,0.038068,0.001205,-0.014261,0.263843
OverallQual,0.032628,0.176561,0.105806,1.000000,-0.091932,0.572323,0.550684,0.407252,0.239666,-0.059119,...,0.238923,0.308819,-0.113937,0.030371,0.064886,0.065166,-0.031406,0.070815,-0.027347,0.790982
OverallCond,-0.059316,-0.053457,-0.005636,-0.091932,1.000000,-0.375983,0.073741,-0.125694,-0.046231,0.040229,...,-0.003334,-0.032589,0.070356,0.025504,0.054811,-0.001985,0.068777,-0.003511,0.043950,-0.077856
YearBuilt,0.027850,0.036853,0.014228,0.572323,-0.375983,1.000000,0.592855,0.311600,0.249503,-0.049107,...,0.224880,0.188686,-0.387268,0.031355,-0.050364,0.004950,-0.034383,0.012398,-0.013618,0.522897
YearRemodAdd,0.040581,0.078686,0.013788,0.550684,0.073741,0.592855,1.000000,0.176529,0.128451,-0.067759,...,0.205726,0.226298,-0.193919,0.045286,-0.038740,0.005829,-0.010286,0.021490,0.035743,0.507101
MasVnrArea,0.023573,0.105010,0.103321,0.407252,-0.125694,0.311600,0.176529,1.000000,0.261256,-0.071330,...,0.159991,0.122528,-0.109907,0.019144,0.062248,0.011928,-0.029512,-0.006723,-0.008317,0.472614
BsmtFinSF1,-0.069836,0.076670,0.214103,0.239666,-0.046231,0.249503,0.128451,0.261256,1.000000,-0.050117,...,0.204306,0.111761,-0.102303,0.026451,0.062021,0.140491,0.003571,-0.015727,0.014359,0.386420
BsmtFinSF2,-0.065649,-0.009312,0.111170,-0.059119,0.040229,-0.049107,-0.067759,-0.071330,-0.050117,1.000000,...,0.067898,0.003093,0.036543,-0.029993,0.088871,0.041709,0.004940,-0.015211,0.031706,-0.011378


Для отображения топ корреляция можно использовать следующий код

In [8]:
def get_redundant_pairs(df):
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(df[numeric_columns], 5)) #выведем топ 5 коррелирующих пар

Top Absolute Correlations
GarageCars   GarageArea      0.882475
GrLivArea    TotRmsAbvGrd    0.825489
TotalBsmtSF  1stFlrSF        0.819530
OverallQual  SalePrice       0.790982
GrLivArea    SalePrice       0.708624
dtype: float64


Уберем признак GarageCars, интуитивно кажется что он вычисляется через GarageArea, почему и такая высокая корреляция

In [9]:
df = df.drop('GarageCars',axis=1)
numeric_columns = numeric_columns.drop('GarageCars')

Далее посмотрим квазиконстантные признаки

In [10]:
from sklearn.feature_selection import VarianceThreshold

cutter = VarianceThreshold(threshold=0.1)
cutter.fit(df[numeric_columns])
constant_cols = [x for x in numeric_columns if x not in cutter.get_feature_names_out()]

df[constant_cols]

,BsmtHalfBath,KitchenAbvGr
Id,,
1,0,1
2,1,1
3,0,1
4,0,1
5,0,1
...,...,...
1456,0,1
1457,0,1
1458,0,1


In [11]:
df['KitchenAbvGr'].value_counts()

KitchenAbvGr
1    1392
2      65
3       2
0       1
Name: count, dtype: int64

In [12]:
df['BsmtHalfBath'].value_counts()

BsmtHalfBath
0    1378
1      80
2       2
Name: count, dtype: int64

Обе колонки больше похожи на категориальные, оставляем как есть.

## Категориальные признаки

In [13]:
categorical_columns = df.loc[:,df.dtypes==object].columns
df.describe(include='object')

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
count,1460,1460,91,1460,1460,1460,1460,1460,1460,1460,...,1379,1379,1379,1379,1460,7,281,54,1460,1460
unique,5,2,2,4,4,2,5,3,25,9,...,6,3,5,5,3,3,4,4,9,6
top,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,...,Attchd,Unf,TA,TA,Y,Gd,MnPrv,Shed,WD,Normal
freq,1151,1454,50,925,1311,1459,1052,1382,225,1260,...,870,605,1311,1326,1340,3,157,49,1267,1198


Некоторые колонки содержат пустые значения

In [14]:
df[categorical_columns].isna().sum()

MSZoning            0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType        872
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
BsmtCond           37
BsmtExposure       38
BsmtFinType1       37
BsmtFinType2       38
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
KitchenQual         0
Functional          0
FireplaceQu       690
GarageType         81
GarageFinish       81
GarageQual         81
GarageCond         81
PavedDrive          0
PoolQC           1453
Fence            1179
MiscFeature      1406
SaleType            0
SaleCondition       0
dtype: int64

Напомним, в описании к датасету и колонкам указано, что NaN означает что этот признак у дома отсутствует(нет дороги, нет подвала и тд.)
Везде заменим отсутствующие значения на категорию 'None'

In [15]:
for col in categorical_columns:
    df[col] = df[col].fillna('None')

In [16]:
#Посмотрим данные ещё раз
df.describe(include='object')

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
count,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,...,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460
unique,5,2,3,4,4,2,5,3,25,9,...,7,4,6,6,3,4,5,5,9,6
top,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,...,Attchd,Unf,TA,TA,Y,None,None,None,WD,Normal
freq,1151,1454,1369,925,1311,1459,1052,1382,225,1260,...,870,605,1311,1326,1340,1453,1179,1406,1267,1198


Теперь закодируем колонки,
если в колонке 5 и более уникальных значений, применим Mean target encoding
для остальных применим OneHotEncoding

In [17]:
for col in categorical_columns:
    if df[col].nunique() < 5:
        one_hot = pd.get_dummies(df[col],prefix=col,drop_first=True)
        df = pd.concat((df.drop(col,axis=1), one_hot), axis=1)

    else:
        mean_target = df.groupby(col)['SalePrice'].mean()
        df[col] = df[col].map(mean_target)


In [18]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,LotConfig,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,...,KitchenQual_Gd,KitchenQual_TA,GarageFinish_None,GarageFinish_RFn,GarageFinish_Unf,PavedDrive_P,PavedDrive_Y,PoolQC_Fa,PoolQC_Gd,PoolQC_None
Id,,,,,,,,,,,,,,,,,,,,,
1,60,191004.994787,65.0,8450,176938.047529,197965.773333,184495.492063,181169.405536,185763.807377,210051.764045,...,True,False,False,True,False,False,True,False,False,True
2,20,191004.994787,80.0,9600,177934.574468,238772.727273,142475.481481,181169.405536,185763.807377,175985.477961,...,False,True,False,True,False,False,True,False,False,True
3,60,191004.994787,68.0,11250,176938.047529,197965.773333,184495.492063,181169.405536,185763.807377,210051.764045,...,True,False,False,True,False,False,True,False,False,True
4,70,191004.994787,60.0,9550,181623.425856,210624.725490,184495.492063,181169.405536,185763.807377,210051.764045,...,True,False,False,False,True,False,True,False,False,True
5,60,191004.994787,84.0,14260,177934.574468,335295.317073,184495.492063,181169.405536,185763.807377,210051.764045,...,True,False,False,True,False,False,True,False,False,True


# Обучение модели

In [19]:
#Разделим данные на признаки и таргет
from sklearn.model_selection import train_test_split
X = df.drop('SalePrice',axis=1)
y = df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# Линейная регрессия, без регуляризации
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True)

model = LinearRegression()

cv_result = cross_validate(model,X_train,y_train,
                           scoring='neg_root_mean_squared_error',
                           cv=kf, return_train_score=True)

print(f'На трейне: {np.mean(cv_result["train_score"]).round(3)}')
print(f'На тесте: {np.mean(cv_result["test_score"]).round(3)}')

На трейне: -22915.86
На тесте: -45801.784


In [21]:
# Модель Lasso и запуск через пайплайн
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso


pipe_lasso = Pipeline([('scaler', StandardScaler()),('Lasso', Lasso(max_iter=120000))])

cv_result_lasso = cross_validate(pipe_lasso,X_train,y_train,
                                 scoring='neg_root_mean_squared_error',
                                 cv=kf,return_train_score=True)

print(f'На трейне: {np.mean(cv_result_lasso["train_score"]).round(3)}')
print(f'На тесте: {np.mean(cv_result_lasso["test_score"]).round(3)}')

На трейне: -23097.781
На тесте: -45374.35


In [22]:
# Ridge
from sklearn.linear_model import Ridge

pipe_ridge = Pipeline([('scaler', StandardScaler()),('Ridge', Ridge(max_iter=120000))])

cv_result_ridge = cross_validate(pipe_ridge,X_train,y_train,
                                 scoring='neg_root_mean_squared_error',
                                 cv=kf,return_train_score=True)

print(f'На трейне: {np.mean(cv_result_ridge["train_score"]).round(3)}')
print(f'На тесте: {np.mean(cv_result_ridge["test_score"]).round(3)}')

На трейне: -23022.002
На тесте: -38415.763


In [23]:
#Подбираем альфа параметр для Lasso
from sklearn.model_selection import GridSearchCV


alphas = np.linspace(0.5,10,30)

param_grid = {
    "Lasso__alpha": alphas
}

search_lasso = GridSearchCV(pipe_lasso, param_grid, cv=kf, scoring='neg_root_mean_squared_error')
search_lasso.fit(X_train,y_train)

print(f"Best parameter (CV score={search_lasso.best_score_:.5f}):")
print(search_lasso.best_params_)

Best parameter (CV score=-43580.09682):
{'Lasso__alpha': np.float64(10.0)}


In [24]:
#Подбираем альфа параметр для Ridge
alphas = np.linspace(0.01,10,30)

param_grid = {
    "Ridge__alpha": alphas
}

search_ridge = GridSearchCV(pipe_ridge, param_grid, cv=kf, scoring='neg_root_mean_squared_error')
search_ridge.fit(X_train,y_train)

print(f"Best parameter (CV score={search_ridge.best_score_:.5f}):")
print(search_ridge.best_params_)

Best parameter (CV score=-34002.28961):
{'Ridge__alpha': np.float64(0.6989655172413793)}


In [25]:
#Фиксируем финальный результат на Ridge модели с лучшими параметрами.

pipe_ridge.set_params(Ridge__alpha = search_ridge.best_params_['Ridge__alpha'])



Pipeline(steps=[('scaler', StandardScaler()),
                ('Ridge',
                 Ridge(alpha=np.float64(0.6989655172413793), max_iter=120000))])

In [26]:
cv_result_ridge = cross_validate(pipe_ridge,X_train,y_train,scoring='neg_root_mean_squared_error',cv=kf,return_train_score=True)

print(f'На трейне: {np.mean(cv_result_ridge["train_score"]).round(3)}')
print(f'На тесте: {np.mean(cv_result_ridge["test_score"]).round(3)}')

На трейне: -23182.211
На тесте: -32988.59


In [27]:
pipe_ridge.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('Ridge',
                 Ridge(alpha=np.float64(0.6989655172413793), max_iter=120000))])

In [28]:
pipe_ridge.score(X_test,y_test)

0.8507810838750005

In [29]:
pred = pipe_ridge.predict(X_test)

In [30]:
result_df = pd.DataFrame({'Predicted': pred, 'Actual': y_test})
result_df.head()

,Predicted,Actual
Id,,
893,137364.936655,154500
1106,338422.592947,325000
414,101926.439679,115000
523,166218.156677,159000
1037,320551.373911,315500
